In [8]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import pickle

In [ ]:
# Load trained model
model = load_model("../models/churn_classification_model.h5")

# Load preprocessing objects
with open("../models/StandardScaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("../models/OneHotEncoder_geo.pkl", "rb") as f:
    OneHotEncoder_geo = pickle.load(f)

with open("../models/LabelEncoder_gender.pkl", "rb") as f:
    LabelEncoder_gender = pickle.load(f)


In [11]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [12]:
# one-hot encoding
geo_encoded = OneHotEncoder_geo.transform([[input_data['Geography']]]).toarray()[0]
geo_df = pd.DataFrame([geo_encoded], columns=OneHotEncoder_geo.get_feature_names_out(['Geography']))
input_df = pd.DataFrame([input_data])
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_df], axis=1)

/Users/ankitbajaj/Desktop/deep Learning/Churn Classification/tf-metal-env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [13]:
input_df 

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
## Encode categorical variables
input_df['Gender']=LabelEncoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
# standardize the input data
input_df_scaled = scaler.transform(input_df)    # Standardize the input data
input_df_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
# predict churn
prediction = model.predict(input_df_scaled)
prediction[0][0]

2025-12-23 23:05:45.414308: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step


0.07577076

In [17]:
prediction_proba = prediction[0][0]

In [18]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
